# Agents and Tools

#### ChatGPT Functions

In [2]:
import sqlite3

conn = sqlite3.connect('db.sqlite')

In [3]:
def run_query(query:str):
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        return cursor.fetchall()
    except sqlite3.OperationalError as err:
        return f"The following error occurred: {str(err)}"

In [5]:
from langchain.tools import Tool


run_query_tool = Tool.from_function(
    name="run_sqlite_query",
    description="""
    run a query on the sqlite database
    """,
    func=run_query,

)


In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder
)
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from dotenv import load_dotenv
load_dotenv()


True

In [7]:
chat = ChatOpenAI(
    model='gpt-3.5-turbo',
)

chat_prompt = ChatPromptTemplate(
    messages =[
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

In [9]:
agent = OpenAIFunctionsAgent(
    llm=chat,
    prompt=chat_prompt,
    tools=[run_query_tool]
)

agent_executor = AgentExecutor(
    agent=agent,
    verbose=True,
    tools=[run_query_tool]
)


In [10]:
agent_executor("How many users are in the database?")



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT COUNT(*) FROM users`


[(2000,)]There are 2000 users in the database.

> Finished chain.


{'input': 'How many users are in the database?',
 'output': 'There are 2000 users in the database.'}

In [14]:
agent_executor("How many users provided shipping address?")



> Entering new AgentExecutor chain...

Invoking: `run_sqlite_query` with `SELECT users.name, COUNT(orders.user_id) as orders_count FROM orders JOIN users ON orders.user_id = users.id GROUP BY orders.user_id`


[('Marvin Mejia', 2), ('Vanessa Young', 1), ('Elizabeth Holmes', 3), ('Laura Hunter', 1), ('Darren Mullins', 1), ('Lauren Perry', 1), ('Mark Phillips', 1), ('Christine Jackson', 1), ('Raymond Price', 1), ('Daniel Harris', 3), ('Jennifer Cohen', 2), ('Maria Brown', 1), ('Mary Garcia', 1), ('Arthur Watson', 1), ('Melissa Holland', 1), ('Robert Thomas IV', 1), ('Darren Hernandez', 1), ('Michael Stevens', 1), ('Jesus Villegas', 2), ('Sarah Schaefer', 2), ('Dennis Dean', 2), ('Kimberly Williams', 1), ('Holly Jones', 1), ('Ronald Lee', 1), ('Brandon Lewis', 1), ('Kristin Poole', 1), ('Lisa Mcpherson', 1), ('Angela Larsen', 1), ('Katie Orr', 1), ('Casey Johnson', 1), ('Lori Reynolds', 3), ('Joel Owens', 1), ('Melissa Daniels', 3), ('Samuel Douglas', 1), ('Marco Munoz', 1), ('Kelly Whi

{'input': 'how many orders by users and also fetch names of users?',
 'output': 'There are multiple users who have placed orders. Here are a few users along with the number of orders they have placed:\n\n1. Marvin Mejia - 2 orders\n2. Vanessa Young - 1 order\n3. Elizabeth Holmes - 3 orders\n4. Laura Hunter - 1 order\n5. Darren Mullins - 1 order\n\nThis list continues with more users and their respective order counts.'}